# Step 06: Submit GeoHaz Batch

This notebook submits GeoHaz (geocoding and hazard) jobs to Moody's Risk Modeler.

**Tasks:**
- Retrieve GeoHaz batch from Stage 01
- Review batch configuration and job details
- Submit batch to Moody's API
- Track submission status

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.batch import submit_batch, get_batch_jobs, read_batch
from helpers.database import execute_query
from helpers.irp_integration import IRPClient
from helpers.constants import BatchType

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_06_Submit GeoHaz_Batch.ipynb')

# Display context
ux.header("GeoHaz Batch Submission")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"✓ Step tracking initialized for '{context.step_name}'")

## 2) Retrieve GeoHaz Batch

In [ ]:
# Retrieve GeoHaz batch from Stage_01/Step_03
ux.subheader("Retrieve GeoHaz Batch")

# Query for Stage_01/Step_03 step run
query = """
    SELECT sr.id, sr.step_id, sr.run_num, sr.output_data, sr.completed_ts
    FROM irp_step_run sr
    INNER JOIN irp_step s ON sr.step_id = s.id
    INNER JOIN irp_stage sg ON s.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
      AND sg.stage_num = 1
      AND s.step_num = 3
      AND sr.status = 'COMPLETED'
    ORDER BY sr.completed_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    raise ValueError("Batch creation step not found - please complete Stage_01/Step_03 first")

output_data = result.iloc[0]['output_data']
batches = output_data.get('batches', {})

if BatchType.GEOHAZ not in batches:
    raise ValueError(f"GeoHaz batch not found. Available: {list(batches.keys())}")

geohaz_batch_id = int(batches[BatchType.GEOHAZ])

ux.success(f"✓ Retrieved GeoHaz batch: ID={geohaz_batch_id}")
step.log(f"Retrieved GeoHaz batch: ID={geohaz_batch_id}")

## 3) Review Batch Configuration

In [ ]:
# Verify batch status and display job information
ux.subheader("Verify Batch Status")

# Read batch details
batch = read_batch(geohaz_batch_id)

batch_info = [
    ["Batch ID", batch['id']],
    ["Batch Type", batch['batch_type']],
    ["Status", batch['status']],
    ["Created", batch['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(batch_info, headers=["Property", "Value"])

# Get jobs in batch
jobs = get_batch_jobs(geohaz_batch_id)
job_count = len(jobs)

ux.info(f"\nTotal jobs: {job_count}")

# Show sample configurations
if job_count > 0:
    ux.info("\nGeoHaz Configurations:")
    rows = []
    for job in jobs:
        config_query = "SELECT job_configuration_data FROM irp_job_configuration WHERE id = %s"
        config_result = execute_query(config_query, (job['job_configuration_id'],))
        if not config_result.empty:
            config = config_result.iloc[0]['job_configuration_data']
            rows.append([
                job['id'],
                config.get('Portfolio', 'N/A'),
                config.get('Database', 'N/A'),
                config.get('geocode_version', 'N/A'),
            ])
    ux.table(rows, headers=["Job ID", "Portfolio", "Database", "Geocode Version"])

step.log(f"Verified batch: {job_count} jobs ready for submission")

## 4) Submit Batch to Moody's

In [ ]:
# Submit batch to Moody's API
ux.subheader("Submit Batch to Moody's")

ux.info("")
ux.info("Submission Process:")
ux.info("  - Each job will geocode and apply hazard data to the portfolio")
ux.info("  - Jobs will transition to SUBMITTED status")
ux.info("  - Batch will transition to ACTIVE status")
ux.info("  - This is an asynchronous operation - jobs may take time to complete")
ux.info("")

# Submit
ux.info("\nSubmitting batch...")

# Pass step.step_id to associate batch with this step (not the creation step)
result = submit_batch(geohaz_batch_id, IRPClient(), step_id=step.step_id)

# Display results
ux.success(f"\n✓ Batch submission completed")
ux.info(f"  Submitted: {result['submitted_jobs']} jobs")
ux.info(f"  Status: {result['batch_status']}")

# Check for errors
failed_count = len([j for j in result['jobs'] if 'error' in j])
if failed_count > 0:
    ux.warning(f"\n⚠ {failed_count} job(s) failed to submit")
    for job_result in result['jobs']:
        if 'error' in job_result:
            ux.error(f"  Job {job_result['job_id']}: {job_result['error']}")

step.log(f"Batch submitted: {result['submitted_jobs']} jobs, {failed_count} failed")

## 5) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

# Prepare output data
output_data = {
    'batch_id': geohaz_batch_id,
    'batch_type': batch['batch_type'],
    'batch_status': result['batch_status'],
    'submitted_jobs': result['submitted_jobs'],
    'failed_jobs': failed_count
}

# Check if any jobs failed to submit
if failed_count > 0:
    # Build error message with failed job details
    failed_job_errors = [
        f"Job {j['job_id']}: {j['error']}" 
        for j in result['jobs'] if 'error' in j
    ]
    error_message = f"{failed_count} job(s) failed to submit:\n" + "\n".join(failed_job_errors)
    
    # Note: Teams notification already sent from batch.py for each failed job
    # Mark step as failed in database (skip duplicate notification)
    from helpers.step import update_step_run
    from helpers.constants import StepStatus
    update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)
    
    ux.error("\n" + "="*60)
    ux.error("✗ GEOHAZ BATCH SUBMISSION FAILED")
    ux.error("="*60)
    ux.info(f"\nSubmitted {result['submitted_jobs'] - failed_count} job(s) successfully")
    ux.error(f"Failed: {failed_count} job(s)")
    ux.info("\nReview errors above and resubmit failed jobs")
else:
    # All jobs submitted successfully - complete the step
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("✓ GEOHAZ BATCH SUBMITTED SUCCESSFULLY")
    ux.success("="*60)
    ux.info(f"\nSubmitted {result['submitted_jobs']} job(s) to Moody's API")
    ux.info(f"Batch status: {result['batch_status']}")
    ux.info("\nNext: Proceed to monitoring notebook to track job progress")